In [153]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, resample_apply
from backtesting.test import SMA
from backtesting import Backtest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd 
import math as m
import os, glob

In [154]:
import warnings
warnings.filterwarnings("ignore")

In [155]:
PRED = True
WithClassOne = False
CASH = 1000
COMMISSION = 0.0001

In [156]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/LSTM/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

3C-1H-LSTM_1H_43F_Class_01_Test.csv
3C-1H-LSTM_1H_43F_Class_01_Train.csv
5C-LSTM_1H_42F_Class_01_Test.csv
5C-LSTM_1H_42F_Class_01_Train.csv
C-LSTM_1D_19F_Class_03_Test.csv
C-LSTM_1D_19F_Class_03_Train.csv
C-LSTM_1H_0F_Class_Test_Results_01.csv
C-LSTM_1H_0F_Class_Test_Results_02.csv
C-LSTM_1H_0F_Class_Train_Results_01.csv
C-LSTM_1H_0F_Class_Train_Results_02.csv


In [157]:
index = 2

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
DATASET = pd.read_csv(filenames[index], header=0)
MODEL_NAME = os.path.basename(filenames[index])
print(MODEL_NAME)
print(DATASET.head(20))
print(DATASET.shape)


5C-LSTM_1H_42F_Class_01_Test.csv
       Open     High      Low    Close      Volume  Real_Class  \
0   1.05809  1.05898  1.05769  1.05892   6150.9399         1.0   
1   1.05891  1.05986  1.05847  1.05958  11677.7197        -2.0   
2   1.05957  1.06041  1.05744  1.05765  15785.2197         2.0   
3   1.05765  1.05956  1.05708  1.05935  14816.4902         0.0   
4   1.05934  1.05946  1.05835  1.05927  10641.8301         2.0   
5   1.05925  1.06104  1.05890  1.06081  13180.9697         0.0   
6   1.06082  1.06181  1.06036  1.06089  14419.7803         1.0   
7   1.06092  1.06179  1.06055  1.06139  15307.1904        -2.0   
8   1.06136  1.06153  1.05658  1.05724  25968.0293        -1.0   
9   1.05719  1.05900  1.05668  1.05668  19795.0098         2.0   
10  1.05670  1.05841  1.05662  1.05801  16532.0801        -2.0   
11  1.05799  1.05970  1.05657  1.05687  11651.7695         1.0   
12  1.05688  1.05776  1.05687  1.05708   8330.3096        -1.0   
13  1.05707  1.05717  1.05573  1.05644   87

In [158]:
NumCLASS =  DATASET['Predicted_Class'].value_counts().shape[0]
print(NumCLASS)

5


In [159]:
if NumCLASS == 3:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1 
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
    else:
        DATASET['Position'] = DATASET['Real_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
if NumCLASS == 5:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
    else:
        DATASET['Position'] = DATASET['Real_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
DATASET.head(10)

,Open,High,Low,Close,Volume,Real_Class,Predicted_Class,Position
0,1.05809,1.05898,1.05769,1.05892,6150.9399,1.0,2.0,1.0
1,1.05891,1.05986,1.05847,1.05958,11677.7197,-2.0,2.0,1.0
2,1.05957,1.06041,1.05744,1.05765,15785.2197,2.0,-2.0,-1.0
3,1.05765,1.05956,1.05708,1.05935,14816.4902,0.0,-2.0,-1.0
4,1.05934,1.05946,1.05835,1.05927,10641.8301,2.0,2.0,1.0
5,1.05925,1.06104,1.05890,1.06081,13180.9697,0.0,-2.0,-1.0
6,1.06082,1.06181,1.06036,1.06089,14419.7803,1.0,2.0,1.0
7,1.06092,1.06179,1.06055,1.06139,15307.1904,-2.0,-2.0,-1.0
8,1.06136,1.06153,1.05658,1.05724,25968.0293,-1.0,2.0,1.0
9,1.05719,1.05900,1.05668,1.05668,19795.0098,2.0,-2.0,-1.0


In [160]:
class limits(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        self.Op_Abierta = False
        
    def next(self):
        sig = self.data.Position[-1]
        if not self.Op_Abierta and sig >= self.buy_lim:
            self.buy()
            self.Op_Abierta = True
        if self.Op_Abierta and sig <= self.sell_lim:
            self.sell()
            self.Op_Abierta = False

In [161]:
class limits2(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        
    def next(self):
        sig = self.data.Position[-1]
        if  sig >= self.buy_lim:
            self.buy()
        if  sig <= self.sell_lim:
            self.sell()

In [162]:
class Position(SignalStrategy):
    def init(self):
        global df
        super().init()  
        self.set_signal(self.data.Position)

In [163]:
DATASET['Predicted_Class'].value_counts()

 2.0    10394
-2.0     5179
-1.0     2965
 0.0     1624
 1.0      142
Name: Predicted_Class, dtype: int64

In [164]:
DATASET['Real_Class'].value_counts()

 0.0    5593
 1.0    4268
-1.0    4141
-2.0    3168
 2.0    3134
Name: Real_Class, dtype: int64

In [165]:
STRATEGY_NAME = limits
print(STRATEGY_NAME.__name__)

limits


In [166]:
bt = Backtest(DATASET, STRATEGY_NAME, cash=CASH, commission=COMMISSION)#trade_on_close=False
bt.run()

Start                               0
End                             20303
Duration                        20303
Exposure [%]                  99.9704
Equity Final [$]              375.246
Equity Peak [$]               1013.39
Return [%]                   -62.4754
Buy & Hold Return [%]         4.97299
Max. Drawdown [%]            -63.0273
Avg. Drawdown [%]           -0.444939
Max. Drawdown Duration            124
Avg. Drawdown Duration           45.2
# Trades                         6764
Win Rate [%]                  42.4157
Best Trade [%]                2.04039
Worst Trade [%]              -1.70302
Avg. Trade [%]             -0.0144608
Max. Trade Duration                68
Avg. Trade Duration           3.00074
Expectancy [%]              0.0924711
SQN                           -6.4598
Sharpe Ratio               -0.0922134
Sortino Ratio               -0.113933
Calmar Ratio             -0.000229438
_strategy                      limits
dtype: object

In [167]:
bt.plot()

In [168]:
data = {'Model_Name':[MODEL_NAME], 'PRED':[PRED], 'Strategy_Class':[STRATEGY_NAME.__name__], 'Equity_Final_$':[bt._results['Equity Final [$]']],
       'Equity_Peak_$':[bt._results['Equity Peak [$]']],'Return_%':[bt._results['Return [%]']],'#_Trades':[bt._results['# Trades']],
       'Win_Rate':[bt._results['Win Rate [%]']],'Commission':[COMMISSION],'Cash':[CASH]}
Results = pd.DataFrame(data)
print(Results)

                         Model_Name  PRED Strategy_Class  Equity_Final_$  \
0  5C-LSTM_1H_42F_Class_01_Test.csv  True         limits      375.245582   

   Equity_Peak_$   Return_%  #_Trades  Win_Rate  Commission  Cash  
0    1013.390121 -62.475442    6764.0  42.41573      0.0001  1000  


In [169]:
filename = 'BackTesting_Results.csv'
print(filename)
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/Backtesting/')
filename_path = os.path.join(pwd, r'../../../Results/Backtesting/' + filename)
print(filename_path)

BackTesting_Results.csv
C:\Users\maw\Documents\ITESO\forex_prediction\Jupyter Notebooks\IDI4\backtesting\../../../Results/Backtesting/BackTesting_Results.csv


In [170]:
#LOAD RESULTS .CSV
pwd = os.getcwd()
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

BackTesting_Results.csv


In [171]:
Results_loaded = pd.read_csv(filenames[0], header=0)
print(Results_loaded)

                              Model_Name   PRED Strategy_Class  \
0   3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits   
1   3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits   
2   3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits   
3   3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits   
4    3C-1H-LSTM_1H_43F_Class_01_Test.csv  False         limits   
5    3C-1H-LSTM_1H_43F_Class_01_Test.csv   True         limits   
6      5C-LSTM_1H_42F_Class_01_Train.csv   True         limits   
7      5C-LSTM_1H_42F_Class_01_Train.csv  False         limits   
8        C-LSTM_1D_19F_Class_03_Test.csv  False         limits   
9        C-LSTM_1D_19F_Class_03_Test.csv   True         limits   
10      5C-LSTM_1H_42F_Class_01_Test.csv  False         limits   

    Equity_Final_$  Equity_Peak_$      Return_%  #_Trades   Win_Rate  \
0     2.929920e+07   2.948331e+07  2.929820e+06   18630.0  69.017713   
1     2.929920e+07   2.948331e+07  2.929820e+06   18630.0  69.0

In [172]:
#print(Results)
Results_loaded = Results_loaded.append(Results,ignore_index=True)
print(Results_loaded)

                              Model_Name   PRED Strategy_Class  \
0   3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits   
1   3C-1H-LSTM_1H_43F_Class_01_Train.csv  False         limits   
2   3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits   
3   3C-1H-LSTM_1H_43F_Class_01_Train.csv   True         limits   
4    3C-1H-LSTM_1H_43F_Class_01_Test.csv  False         limits   
5    3C-1H-LSTM_1H_43F_Class_01_Test.csv   True         limits   
6      5C-LSTM_1H_42F_Class_01_Train.csv   True         limits   
7      5C-LSTM_1H_42F_Class_01_Train.csv  False         limits   
8        C-LSTM_1D_19F_Class_03_Test.csv  False         limits   
9        C-LSTM_1D_19F_Class_03_Test.csv   True         limits   
10      5C-LSTM_1H_42F_Class_01_Test.csv  False         limits   
11      5C-LSTM_1H_42F_Class_01_Test.csv   True         limits   

    Equity_Final_$  Equity_Peak_$      Return_%  #_Trades   Win_Rate  \
0     2.929920e+07   2.948331e+07  2.929820e+06   18630.0  69.017713 

In [173]:
Results_loaded.to_csv(filename_path, header=True, index=False)